# Following Code removes background from Image with OPEN CV

In [11]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import sys
from  PIL  import Image

In [12]:
img = cv.imread("D:/Ultrasound/Slices-Images/2019_03_13-M01-1.png", cv.IMREAD_UNCHANGED)
original = img.copy()
print(original.shape)

# resize image
scale_percent = 40 # percent of original size
width = int(original.shape[1] * scale_percent / 100)
height = int(original.shape[0] * scale_percent / 100)
dim = (width, height)
original_resized = cv.resize(original, dim, interpolation = cv.INTER_AREA)

print(original_resized.shape)
cv.imshow('resized image',original_resized)
cv.waitKey(0) # waits until a key is pressed
cv.destroyAllWindows() # destroys the window showing image

(2868, 3129, 4)
(1147, 1251, 4)


In [13]:
edges_Gaussian = cv.GaussianBlur(original_resized, (0, 0), 3)
# https://www.tutorialkart.com/opencv/python/opencv-python-gaussian-image-smoothing/
#print(edges_Gaussian.shape)
#cv.imshow('resized image',edges_Gaussian)
#cv.waitKey(0) # waits until a key is pressed
#cv.destroyAllWindows() # destroys the window showing image

edges_Gaussian_Gray = cv.cvtColor(edges_Gaussian, cv.COLOR_BGR2GRAY)
# https://www.geeksforgeeks.org/python-opencv-cv2-cvtcolor-method/#:~:text=cvtColor()%20method%20is%20used,color%20space%20conversion%20codes%20below.
print(edges_Gaussian_Gray.shape)
cv.imshow('resized image',edges_Gaussian_Gray)
cv.waitKey(0) # waits until a key is pressed
cv.destroyAllWindows() # destroys the window showing image

(1147, 1251)


In [36]:
edges = cv.Canny(edges_Gaussian, 7, 12)
# https://docs.opencv.org/4.x/da/d22/tutorial_py_canny.html
#print(edges.shape)
#cv.imshow('resized image',edges)
#cv.waitKey(0) # waits until a key is pressed
#cv.destroyAllWindows() # destroys the window showing image

_, thresh = cv.threshold(edges, 0, 255, cv.THRESH_BINARY  + cv.THRESH_OTSU) # https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html
kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (1, 1))
mask = cv.morphologyEx(thresh, cv.MORPH_CLOSE, kernel, iterations=1)
print(mask.shape)
cv.imshow('resized image',mask)
cv.waitKey(0) # waits until a key is pressed
cv.destroyAllWindows() # destroys the window showing image

(1147, 1251)


In [37]:
data = mask.tolist()
sys.setrecursionlimit(10**8)
for i in  range(len(data)):
    for j in  range(len(data[i])):
        if data[i][j] !=  255:
            data[i][j] =  -1
        else:
            break
    for j in  range(len(data[i])-1, -1, -1):
        if data[i][j] !=  255:
            data[i][j] =  -1
        else:
            break
image = np.array(data)
image[image !=  -1] =  255
image[image ==  -1] =  0

mask = np.array(image, np.uint8)

print(mask.shape)
cv.imshow('resized image',mask)
cv.waitKey(0) # waits until a key is pressed
cv.destroyAllWindows() # destroys the window showing image

(1147, 1251)


In [38]:
result = cv.bitwise_and(original_resized, original_resized, mask=mask)
result[mask ==  0] =  0
print(result.shape)
cv.imshow('resized image',result)
cv.waitKey(0) # waits until a key is pressed
cv.destroyAllWindows() # destroys the window showing image

(1147, 1251, 4)


In [39]:
cv.imwrite('Slices/bg.png', result)

img = Image.open('Slices/bg.png')
img.convert("RGBA")
datas = img.getdata()

newData = []
for item in datas:
    if item[0] ==  0  and item[1] ==  0  and item[2] ==  0:
        newData.append((0, 0, 0, 255))
    else:
        newData.append(item)

img.putdata(newData)
img.save("Slices/img.png", "PNG")